In [2]:
%pip install typing_extensions


  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Default State

In [7]:
from typing_extensions import TypedDict


# Default: No reducers, this will just update the messages previously held in the state
class State(TypedDict):
    foo: int
    bar: list[str]



## Operator add used as the reducer for the second key


In [10]:
# Here we are going to use the Annotated function to specify an operator add.

from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: int
    bar: Annotated[list[str], add]

# This would append the list whenever there's a change in the state of the second key


### Working with messages in GraphState with modern LLMs

Modern LLM providers like Langchain accept a list of Messages with their ChatModel's.

- AIMessage
- HumanMessage

If you wanted to simply append messages to the existing list, you could use operator.add as a reducer.

### However, you might also want to manually update messages in your graph state (e.g. human-in-the-loop). 

If you were to use operator.add, the manual state updates you send to the graph would be appended to the existing list of messages, instead of updating existing messages. To avoid that, you need a reducer that can keep track of message IDs and overwrite existing messages, if updated. To achieve this, you can use the prebuilt add_messages function. For brand new messages, it will simply append to existing list, but it will also handle the updates for existing messages correctly.


### The add_messages function will also try to deserialize messages into LangChain Message objects whenever a state update is received on the messages channel.


### MessagesState¶

Since having a list of messages in your state is so common, there exists a prebuilt state called MessagesState which makes it easy to use messages. MessagesState is defined with a single messages key which is a list of AnyMessage objects and uses the add_messages reducer. Typically, there is more state to track than just messages, so we see people subclass this state and add more fields, like:



In [14]:
from langgraph.graph import MessagesState

class State(MessagesState):
    documents: list[str]